In [ ]:
###
# PWM control for RPi
# w/ alternating RPM
###

import os     # importing os library to communicate with the system
import time   # importing time library to make Rpi wait
import sched  #
os.system ("sudo killall pigpiod")
time.sleep(2) # Wait to make sure pigpiod process is not running
os.system ("sudo pigpiod") # Launching GPIO library
time.sleep(1) # delay prevents timing error
import pigpio # importing GPIO library

global opt
opt = 1

scheduler = sched.scheduler(time.time, time.sleep)

ESC = 18  # GPIO pin connected to ESC 

pi = pigpio.pi();
pi.set_servo_pulsewidth(ESC, 0) 

max_value = 2000 # ESC's max value 
min_value = 700  # ESC's min value
##print ("Select option #:")
##print ("1 : Manual")
##print ("2 : Calibrate")
##print ("3 : Arm")
##print ("4 : Increment Control")
##print ("5 : Alternate RPM")
##print ("6 : Stop")

def manual_drive(): # Function to program ESC if required
    print ("Manual selected -> give a value between 0 and max value")
    while True:
        inp = input()
        if inp == "stop":
            stop()
            break
        elif inp == "control":
            control()
            break
        elif inp == "arm":
            arm()
            break	
        else:
            pi.set_servo_pulsewidth(ESC,inp)
                
def calibrate():   # Auto calibration procedure of a normal ESC
    pi.set_servo_pulsewidth(ESC, 0)
    print("Disconnect the battery and press Enter")
    inp = input()
    pi.set_servo_pulsewidth(ESC, max_value)
    print("Connect the battery press Enter")
    inp = input()     
    pi.set_servo_pulsewidth(ESC, min_value)
    print (".")
    time.sleep(7)
    print ("..")
    time.sleep (5)
    print ("...")
    pi.set_servo_pulsewidth(ESC, 0)
    time.sleep(2)
    print ("....")
    pi.set_servo_pulsewidth(ESC, min_value)
    time.sleep(1)
    print (".....")
    main() # changable to any fn
            
def control(): 
    print ("Starting the motor, restart by giving 'x'")
    time.sleep(1)
    speed = 1500    # change your speed if you want to.... it should be between 700 - 2000
    print ("Controls - a to decrease speed & d to increase speed OR q to decrease a lot of speed & e to increase a lot of speed")
    while True:
        pi.set_servo_pulsewidth(ESC, speed)
        inp = input()
        
        if inp == "q":
            speed -= 100    # decrementing the speed by a lot
            print ("speed -100")
        elif inp == "e":    
            speed += 100    # incrementing the speed by a lot
            print ("speed +100")
        elif inp == "d":
            speed += 10     # incrementing the speed 
            print ("speed +10")
        elif inp == "a":
            speed -= 10     # decrementing the speed
            print ("speed -10")
        elif inp == "stop":
            stop()          # stop function
            break
        elif inp == "manual":
            manual_drive()
            break
        elif inp == "arm":
                arm()
                break	
        else:
            print ("Press a,q,d or e")
            
def arm(): #This is the arming procedure of an ESC 
    print ("Connect the battery and press Enter")
    inp = input()    
    pi.set_servo_pulsewidth(ESC, 0)
    time.sleep(1)
    pi.set_servo_pulsewidth(ESC, max_value)
    time.sleep(1)
    pi.set_servo_pulsewidth(ESC, min_value)
    time.sleep(1)
    control() 
        
def stop(): # Stop Pi actions for ESC.
    pi.set_servo_pulsewidth(ESC, 0)
    pi.stop()

def new_timed_call(calls_per_second, callback, *args, **kw):
    period = 1.0 / calls_per_second
    def reload():
        callback(*args, **kw)
        scheduler.enter(period, 0, reload, ())
    scheduler.enter(period, 0, reload, ())

def changeRPM():
    global opt
    if opt == 1:
        pi.set_servo_pulsewidth(ESC,1200)  # pulsewidth 1
        # print("1200") # print for testing
        opt = 2
    elif opt == 2:
        pi.set_servo_pulsewidth(ESC,1210)  # pulsewidth 2
        # print("1250") # print for testing
        opt = 1
    
    ##########################

def altRPM():
    try:
        print(".....Alternating RPM.....")
        print(" swaps between PWM lengths of 800 and 1500")
        print("10 times per second")
        pi.set_servo_pulsewidth(ESC, 0)
        print("Press Enter to start Calibration")
        inp = input()
        print("Setting max value to ", max_value)
        pi.set_servo_pulsewidth(ESC, max_value)
        print ("calibrating..")
        time.sleep(5)
        pi.set_servo_pulsewidth(ESC, 0)
        time.sleep(2)
        print("Setting min value to ", min_value)
        pi.set_servo_pulsewidth(ESC, min_value)
        time.sleep(2)
        pi.set_servo_pulsewidth(ESC, 0)
        print("Done.")
        print("Press Enter to continue.")
        inp = input()     
        time.sleep(1)
        print ("Starting RPM alternation")
        print ("Ctrl-C to stop")
        changeRPM()
        new_timed_call(10, changeRPM)  # Change RPM xxx times a second
        scheduler.run()
    except:
        print ("Interrupted -> Stopping")
        print ("Error or Keyboard interrupt:")
        pi.set_servo_pulsewidth(ESC, 0)
        pi.stop()
        raise

def main(): # Main program
   
    time.sleep(1)
    pi.set_servo_pulsewidth(ESC, 0) 

    print ("Select option #:")
    print ("1 : Manual")
    print ("2 : Calibrate")
    print ("3 : Arm")
    print ("4 : Increment Control")
    print ("5 : Alternate RPM")
    print ("6 : Stop")

    inp = input()
    if int(inp) == 1:
        manual_drive()
    elif int(inp) == 2:
        calibrate()
    elif int(inp) == 3:
        arm()
    elif int(inp) == 4:
        control()
    elif int(inp) == 5:
        altRPM()
    elif int(inp) == 6:
        stop()
    
    else :
        print ("Error, restarting in 3..")
        time.sleep (3)
        main()
main()


In [ ]:
def loopAlternate(): # Runs for about 5 seconds
  # Using a period of T = 1/25
  pi.set_servo_pulsewidth(ESC,2000)
  time.sleep(2)
  pi.set_servo_pulsewidth(ESC,700)
  time.sleep(2)
  print("Starting loop")
  arr = np.zeros(1000)
  arr[0:299] = 1200 #intitialize
  for j in range(300,len(arr-1),2):
    arr[j]=1200
  for k in range(301,len(arr-1),2):
    arr[k]=1250 
  pauseLength = 0.01 # 10 milliseconds -> 1/2 cycle
  for i in arr: # 0.04 * 125 = 5 seconds
    pi.set_servo_pulsewidth(ESC,i)
    print(i)
    time.sleep(pauseLength) # 20 ms
   # pi.set_servo_pulsewidth(ESC,1250)
   # time.sleep(pauseLength) # 20 ms
    
  stop()

In [ ]:
## Calculating RPM

kv = 10000 # RPM / V
poles = 6 # pairs
erpm = 60000
rpm = erpm / poles
print("RPM =", rpm)


RPM = 10000.0


In [ ]:
dat = [0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1]